In [1]:
import pandas as pd
from preprocess_data import *
import argparse 
from preprocess_data import *
import os
from string import punctuation
import numpy as np
import pandas as pd
import re 
from collections import Counter

## Read in full dataset, drop some rows to make it smaller

In [3]:
data = read_from_uris('/Users/myradeng/Documents/CS230/cs230-final/all-the-news.csv')

/Users/myradeng/Documents/CS230/cs230-final/preprocess_data.py:29: DtypeWarning: Columns (1,3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  data_1 = pd.read_csv(uri1)


In [4]:
right = ['Fox News', ]
left = ['Buzzfeed News', 'CNN', 'New Yorker', 'New Republic', 'The New York Times', 'Vox', 'Vice News', 'Washington Post']
moderate = ['Axios', 'Business Insider', 'The Hill', 'Politico', 'CNBC', 'Economist', 'Politico', 'Reuters', 'The Hill']

In [66]:
right_data = data[data['publication'] == 'Fox News']
left_data = data[data['publication'].isin(left)]
moderate_data = data[data['publication'].isin(moderate)]

In [67]:
balanced_article_count = min(len(right_data), len(left_data), len(moderate_data))

In [68]:
balanced_article_count

20144

In [69]:
def modify_length(data, new_length):
    remove_n = len(data) - new_length
    drop_indices = np.random.choice(data.index, remove_n, replace=False)
    data_cropped = data.drop(drop_indices)
    return data_cropped

In [92]:
right_data_cropped = modify_length(right_data, int(balanced_article_count / 7))

In [93]:
left_data_cropped = modify_length(left_data, int(balanced_article_count / 8))

In [94]:
moderate_data_cropped = modify_length(moderate_data, int(balanced_article_count / 4))

In [95]:
data_balanced = pd.concat([right_data_cropped, moderate_data_cropped, left_data_cropped], axis=0)

In [158]:
right_data_cropped['article'][1680753]

"close Video Pence, McConnell welcome Brett Kavanaugh to Capitol Hill Vice president praises the credentials and character of President Trump's nominee to replace retiring Supreme Court Justice Anthony Kennedy. If we were to pick one symbol of freedom in today’s politically charged America, it must be the gun. The constitutional right to own and carry a firearm sets America apart from the world and puts us ahead of every other nation in terms of individual freedom. It is therefore fitting that both sides of the gun issue are voicing some of the clearest comments on President Trump’s nomination Monday of Judge Brett Kavanaugh – currently serving on the U.S. Circuit Court of Appeals for the District of Columbia – to become an associate justice of the U.S. Supreme Court. Former New York City Mayor Michael Bloomberg’s anti-gun rights group, Everytown for Gun Safety, essentially declared this judge would end our civil society, saying it a statement: “Judge Kavanaugh has applied an extreme a

## Process data

In [96]:
data_balanced = data_balanced.rename(columns={"article": "content"})

In [47]:
data_balanced['content'] = data_balanced['content'].str.replace("&apos", "'")

In [97]:
data_balanced_processed = process_data(data_balanced)

1


/var/folders/zp/rj2crtv10fnb7ytz_gytl7pw0000gn/T/ipykernel_1319/610411353.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  data['content'] = data['content'].str.replace(' +', ' ')


2
3
4
Male pronoun count: 57457
Female pronoun count: 24595


In [98]:
data_balanced_processed = data_balanced_processed.drop(columns=['Unnamed: 0.1'])

In [99]:
print(len(data_balanced_processed[data_balanced_processed['publication'] == 'Fox News']),
      len(data_balanced_processed[data_balanced_processed['publication'].isin(left)]),
      len(data_balanced_processed[data_balanced_processed['publication'].isin(moderate)]))

105769 119194 112333


In [100]:
data_balanced_processed["content"][1748]

'CHARLES HURT, FOX NEWS CONTRIBUTOR: Well, certainly the report reveals something that a lot of it had known it for a long time.'

### Drop some malformed rows

In [102]:
indices = []
count = 0
for row_index, row in data_balanced_processed.iterrows():
    if re.search(r"\\r", str(row)):
        print(row)               #Return type is pandas.Series
        indices.append(row_index)
        count +=1

date                                          2018-05-15 00:00:00
year                                                         2018
month                                                         5.0
day                                                            15
author                                             Tyler McCarthy
title           'Roseanne' recap: drug addiction rocks the Con...
content         (ABC/Adam Rose)\rROSEANNE BARR, JOHN GOODMAN\n...
url             https://www.foxnews.com/entertainment/roseanne...
section                                                     Recap
publication                                              Fox News
label                                                           0
content_list    [(ABC/Adam, Rose), ROSEANNE, BARR,, JOHN, GOOD...
Name: 12035, dtype: object
date                                          2018-05-26 00:00:00
year                                                         2018
month                                            

date                                          2016-01-28 01:00:02
year                                                         2016
month                                                         1.0
day                                                            28
author                                                  Dara Lind
title           Vox Sentences: What if they held a debate and ...
content         newsletter-li {\r margin: 20px 0;\r font-size: 1.
url             https://www.vox.com/2016/1/27/10854748/vox-sen...
section                                                       NaN
publication                                                   Vox
label                                                           0
content_list    [newsletter-li, {, margin:, 20px, 0;, font-siz...
Name: 233738, dtype: object
date                                          2016-01-28 01:00:02
year                                                         2016
month                                           

In [103]:
indices

[12035,
 24580,
 37800,
 229935,
 229936,
 229937,
 229938,
 229939,
 229940,
 229941,
 229942,
 229943,
 229944,
 229945,
 229946,
 229947,
 229948,
 229949,
 229950,
 229951,
 229952,
 229953,
 229954,
 229955,
 230687,
 230688,
 230689,
 230690,
 230691,
 230692,
 230693,
 230694,
 230695,
 230696,
 230697,
 230698,
 230699,
 230700,
 230701,
 230702,
 230703,
 230704,
 230705,
 230706,
 230707,
 233738,
 233739,
 233740,
 233741,
 233742,
 233743,
 233744,
 233745,
 233746,
 233747,
 233748,
 233749,
 233750,
 233751,
 233752,
 233753,
 233754,
 233755,
 233756,
 233757,
 233758]

In [104]:
data_balanced_processed.drop(index=indices,inplace=True)

In [154]:
data_balanced_processed[['date', 'author', 'title', 'content', 'url', 'section', 'publication']].tail(50)

,date,author,title,content,url,section,publication
348362,2020-03-17 09:37:46,David Leonhardt,Opinion | ‘More Severe Than the Great Recession’,"Shortly after the 9/11 attacks, with a deep ec...",https://www.nytimes.com/2020/03/17/opinion/cor...,opinion,The New York Times
348363,2020-03-17 09:37:46,David Leonhardt,Opinion | ‘More Severe Than the Great Recession’,Bush asked Americans to do their part to help ...,https://www.nytimes.com/2020/03/17/opinion/cor...,opinion,The New York Times
348364,2020-03-17 09:37:46,David Leonhardt,Opinion | ‘More Severe Than the Great Recession’,"“Get down to Disney World in Florida,” Bush said.",https://www.nytimes.com/2020/03/17/opinion/cor...,opinion,The New York Times
348365,2020-03-17 09:37:46,David Leonhardt,Opinion | ‘More Severe Than the Great Recession’,"“Take your families and enjoy life, the way we...",https://www.nytimes.com/2020/03/17/opinion/cor...,opinion,The New York Times
348366,2020-03-17 09:37:46,David Leonhardt,Opinion | ‘More Severe Than the Great Recession’,” That kind of advice obviously would not work...,https://www.nytimes.com/2020/03/17/opinion/cor...,opinion,The New York Times
348367,2020-03-17 09:37:46,David Leonhardt,Opinion | ‘More Severe Than the Great Recession’,The only way to reduce the spread of the coron...,https://www.nytimes.com/2020/03/17/opinion/cor...,opinion,The New York Times
348368,2020-03-17 09:37:46,David Leonhardt,Opinion | ‘More Severe Than the Great Recession’,"(If anything, Disney damaged public health by ...",https://www.nytimes.com/2020/03/17/opinion/cor...,opinion,The New York Times
348369,2020-03-17 09:37:46,David Leonhardt,Opinion | ‘More Severe Than the Great Recession’,) Yet the federal government clearly needs to ...,https://www.nytimes.com/2020/03/17/opinion/cor...,opinion,The New York Times
348370,2020-03-17 09:37:46,David Leonhardt,Opinion | ‘More Severe Than the Great Recession’,"A recession has already begun, in all likeliho...",https://www.nytimes.com/2020/03/17/opinion/cor...,opinion,The New York Times
348371,2020-03-17 09:37:46,David Leonhardt,Opinion | ‘More Severe Than the Great Recession’,Initial forecasts suggest the economy could sh...,https://www.nytimes.com/2020/03/17/opinion/cor...,opinion,The New York Times


In [106]:
train, dev, test = write_data_to_dir(data_balanced_processed, '/Users/myradeng/Documents/CS230/cs230-final/data-pt-3')

Created new directory at: /Users/myradeng/Documents/CS230/cs230-final/data-pt-3
Writing data to: /Users/myradeng/Documents/CS230/cs230-final/data-pt-3


In [109]:
test

,date,year,month,day,author,title,content,url,section,publication,label,content_list
201308,2019-08-21 00:00:00,2019,8.0,21,Jonathan Landay,U.S. imposes sanctions on three Chinese accuse...,WASHINGTON (Reuters) - The YOU.,https://www.reuters.com/article/us-usa-china-f...,U.S. Legal News,Reuters,0,"[WASHINGTON, (Reuters), -, The, YOU.]"
169026,2018-12-20,2018,12.0,20,NaN,World Bank on China economy: expect growth to ...,Consumption will remain the main driver of Chi...,https://www.cnbc.com/2018/12/20/world-bank-on-...,China Economy,CNBC,0,"[Consumption, will, remain, the, main, driver,..."
248057,2017-02-23 00:00:00,2017,2.0,23,Jim Sciutto,FBI refused White House request to knock down ...,"""We did not try to knock the story down.",https://www.cnn.com/2017/02/23/politics/fbi-re...,politics,CNN,0,"[""We, did, not, try, to, knock, the, story, do..."
216144,2019-12-20 00:00:00,2019,12.0,20,"Aftab Ahmed, Zeba Siddiqui","From housewives to hijab-clad students, women ...","In the past, women have played a prominent rol...",https://www.reuters.com/article/india-citizens...,World News,Reuters,0,"[In, the, past,, women, have, played, a, promi..."
47981,2018-06-13 00:00:00,2018,6.0,13,Fox News,"After Kim meeting, Trump says fake news is the...",But it is also very special for me and for my ...,https://www.foxnews.com/transcript/after-kim-m...,TRANSCRIPT,Fox News,0,"[But, it, is, also, very, special, for, me, an..."
...,...,...,...,...,...,...,...,...,...,...,...,...
126213,2016-06-08 00:00:00,2016,6.0,8,Dan Levine,Apple case against Samsung should go back to l...,"Following a 2012 jury trial, Samsung was order...",http://www.reuters.com/article/apple-samsung-e...,Business News,Reuters,0,"[Following, a, 2012, jury, trial,, Samsung, wa..."
314779,2018-10-28 08:17:57,2018,10.0,28,"Rory Smith, Iliana Magra","Vichai Srivaddhanaprabha, Leicester City Owner...",He then turned it into a Premier League champi...,https://www.nytimes.com/2018/10/28/world/europ...,world,The New York Times,0,"[He, then, turned, it, into, a, Premier, Leagu..."
158510,2017-08-01,2017,8.0,1,NaN,Companies top athletes are investing in,"""When I met the Blue Jeans team, I was busy st...",https://www.cnbc.com/2017/08/01/companies-top-...,NaN,CNBC,0,"[""When, I, met, the, Blue, Jeans, team,, I, wa..."
121475,2017-07-19,2017,7.0,19,NaN,Minnesota Twins - PlayerWatch,He is now secured 11 straight save opportuniti...,https://www.reuters.com/article/bbo-twins-play...,US MLB,Reuters,0,"[He, is, now, secured, 11, straight, save, opp..."


### Sanity checks

In [240]:
from datasets import load_dataset
dataset = load_dataset("myradeng/cs-230-news-v2")

Using custom data configuration myradeng--cs-230-news-v2-8c9da6f3f3e55055


Extracting data files: 100%|█████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 249.09it/s]


Dataset csv downloaded and prepared to /Users/myradeng/.cache/huggingface/datasets/myradeng___csv/myradeng--cs-230-news-v2-8c9da6f3f3e55055/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 170.64it/s]


In [128]:
test = pd.read_csv('/Users/myradeng/Documents/CS230/cs230-final/data-pt-3/test.csv')

In [129]:
right_data = test[test['publication'] == 'Fox News']
left_data = test[test['publication'].isin(left)]
moderate_data = test[test['publication'].isin(moderate)]

In [130]:
train

,date,year,month,day,author,title,content,url,section,publication,label,content_list
318598,2019-02-28 17:00:04,2019,2.0,28,Sean L. McCarthy,6 Comedy Shows to Catch in N.Y.C. This Weekend,Ticket proceeds will go to the American Bird C...,https://www.nytimes.com/2019/02/28/arts/nyc-th...,arts,The New York Times,0,"[Ticket, proceeds, will, go, to, the, American..."
198634,2019-02-03 00:00:00,2019,2.0,3,"David Beavers,Bryan Bender",Trump determined to 'bring our folks back home...,S.,https://www.politico.com/story/2019/02/03/trum...,NaN,Politico,0,[S.]
196661,2016-08-01,2016,8.0,1,"POLITICO Staff,Glenn Thrush",Full transcript: POLITICO's Glenn Thrush inter...,"How have you, kind of, like, seen him change o...",https://www.politico.com/story/2016/08/full-tr...,NaN,Politico,0,"[How, have, you,, kind, of,, like,, seen, him,..."
159946,2017-10-20,2017,10.0,20,NaN,Tense earnings call caps rocky six months for ...,"Executives on the call, including Munoz, repea...",https://www.cnbc.com/2017/10/20/tense-earnings...,Airlines,CNBC,0,"[Executives, on, the, call,, including, Munoz,..."
83047,2018-07-21,2018,7.0,21,Stephen Sorace,Jamie Lee Curtis thanked by emotional Comic-Co...,He explained how he grabbed the needles out of...,https://www.foxnews.com/entertainment/jamie-le...,Movies,Fox News,0,"[He, explained, how, he, grabbed, the, needles..."
...,...,...,...,...,...,...,...,...,...,...,...,...
256217,2018-12-17,2018,12.0,17,Jeremy Herb,"Comey slams Trump, 'shameful' House Republican...","In the long run, it will make no difference be...",https://www.cnn.com/2018/12/17/politics/james-...,politics,CNN,0,"[In, the, long, run,, it, will, make, no, diff..."
104605,2018-08-09,2018,8.0,9,Fox News,Midterms exposing divide in Democratic Party?,"CHAFFETZ: Oh my goodness, we are switching top...",https://www.foxnews.com/transcript/midterms-ex...,TRANSCRIPT,Fox News,0,"[CHAFFETZ:, Oh, my, goodness,, we, are, switch..."
29228,2018-05-31 00:00:00,2018,5.0,31,Fox News,"Gowdy faces backlash over remarks about FBI, T...",HANNITY: No.,https://www.foxnews.com/transcript/gowdy-faces...,TRANSCRIPT,Fox News,0,"[HANNITY:, No.]"
330644,2019-08-07 05:00:13,2019,8.0,7,Mitch Smith,Policing: What Changed (and Didn’t) Since Mich...,"Since the beginning of 2015, on-duty law enfor...",https://www.nytimes.com/2019/08/07/us/racism-f...,us,The New York Times,0,"[Since, the, beginning, of, 2015,, on-duty, la..."


In [123]:
#right_data = right_data.drop(columns="Unnamed: 0")
left_data = left_data.drop(columns="Unnamed: 0")
moderate_data = moderate_data.drop(columns="Unnamed: 0")

In [125]:
right_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9489 entries, 0 to 32200
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          9489 non-null   object 
 1   year          9489 non-null   int64  
 2   month         9489 non-null   float64
 3   day           9489 non-null   int64  
 4   author        9489 non-null   object 
 5   title         9489 non-null   object 
 6   content       9489 non-null   object 
 7   url           9489 non-null   object 
 8   section       9489 non-null   object 
 9   publication   9489 non-null   object 
 10  label         9489 non-null   int64  
 11  content_list  9489 non-null   object 
dtypes: float64(1), int64(3), object(8)
memory usage: 963.7+ KB


In [132]:
right_data.to_csv('/Users/myradeng/Documents/CS230/cs230-final/data-pt-3/test-conservative.csv')
left_data.to_csv('/Users/myradeng/Documents/CS230/cs230-final/data-pt-3/test-liberal.csv')
moderate_data.to_csv('/Users/myradeng/Documents/CS230/cs230-final/data-pt-3/test-moderate.csv')

In [131]:
print(len(right_data), len(left_data), len(moderate_data))

10500 11981 11242


## Perform CDA on training data only 

In [9]:
# With original sentences + without 

In [69]:
train = pd.read_csv('/Users/myradeng/Documents/CS230/cs230-final/data-pt-3/train.csv')

In [70]:
train_CDA_only = train.copy()

In [12]:
# From winobias repo

In [71]:
swap_words_file = open('/Users/myradeng/Documents/CS230/cs230-final/data-pt-3/swap_words.txt', 'r')
swap_words = swap_words_file.readlines()
swap_words_dict = dict()
count = 0
for line in swap_words:
    count += 1
    line = line.split()
    swap_words_dict[line[0]] = line[1]
    swap_words_dict[line[1]] = line[0]
#     print(line)
#     print("Line{}: {}".format(count, line.strip()))

In [73]:
swap_words_dict['his'] = 'hers'
swap_words_dict['hers'] = 'his'

In [75]:
def do_swap(input_text):
    gender_identifiers = swap_words_dict.keys()
    #print(gender_identifiers)
    input_text_list = input_text.lower().split()
    for idx in range(0, len(input_text_list)): 
        if input_text_list[idx] in gender_identifiers:
            input_text_list[idx] = swap_words_dict[input_text_list[idx]]
    return " ".join(input_text_list)

In [76]:
count = 0
for idx, row in train_CDA_only.iterrows():
    train_CDA_only.loc[idx, "content"] = do_swap(row["content"])

In [86]:
len(train_CDA_only)

269784

In [88]:
len(train)

269784

In [91]:
train_CDA_and_orig = pd.concat([train, train_CDA_only])

In [94]:
train_CDA_and_orig = train_CDA_and_orig.drop(columns='Unnamed: 0')

In [96]:
train_CDA_and_orig.to_csv('/Users/myradeng/Documents/CS230/cs230-final/data-pt-3/train_CDA_and_orig.csv')

In [97]:
train_CDA_only.to_csv('/Users/myradeng/Documents/CS230/cs230-final/data-pt-3/train_CDA_only.csv')

## Functions

In [51]:
import contractions

In [101]:
def write_data_to_dir(data, output_dir):
    # Shuffle and split data intro train, dev, test 
    data = shuffle_data(data)
    split_1 = int(0.8 * len(data))
    split_2 = int(0.9 * len(data))
    train_data = data[:split_1]
    dev_data = data[split_1:split_2]
    test_data = data[split_2:]

    # Write data out to three separate files under output dir 
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print('Created new directory at: ' + output_dir)
    print('Writing data to: ' + output_dir)
    train_data.to_csv(output_dir + '/train.csv')
    dev_data.to_csv(output_dir + '/dev.csv')
    test_data.to_csv(output_dir + '/test.csv')
    return train_data, dev_data, test_data

In [49]:
def process_data(data):
    #output_dir = '/Users/myradeng/Documents/CS230/cs-230-final/' + data_type + '-data'
    filter_gender = False
    proportion_gender = False
    ratio = 1.3
    
    print("1")
    data = data.drop(columns=['Unnamed: 0'])
    data = data[data['content'].notna()]
    data['content'] = data['content'].str.replace(' +', ' ')
    data['content'] = data['content'].str.replace("&apos", "'")
    data['label'] = 0
    

    print("2")
    # Split data entries into separate sentences and remove empty strings
    data = split_df_list(data, 'content', '.')
    data = data[data['content'].str.len() > 0]
    print("3")
    data['content'] = np.where(data['content'].str.endswith('.'), data['content'], data['content'] + '.')
    data['content'] = data['content'].str.strip()
    data['content'] = data['content'].astype(str)
    data['content'] = data['content'].apply(lambda x: contractions.fix(x))

    data['content' + "_list"] = data['content'].str.split()
    print("4")
    # (Configurable) Filter for single instance pronoun entries
    if filter_gender:
        data['pronoun_count'] = data.apply(lambda row: sum([item in row['content_list'] for item in woman_keywords + man_keywords]), axis=1)
        data = data[data['pronoun_count'] == 1]

    # Count initial gender proportions
    female_pronoun_count, male_pronoun_count = count_gender_entries(data)
    print('Male pronoun count: ' + str(male_pronoun_count))
    print('Female pronoun count: ' + str(female_pronoun_count))

    # Ensure approx equal split between male / female gender pronouns in training data
    if proportion_gender:
        data = proportion_gender_entries(data, male_pronoun_count, female_pronoun_count, ratio)
        female_pronoun_count, male_pronoun_count = count_gender_entries(data)
        print('Male pronoun count after rebalancing: ' + str(male_pronoun_count))
        print('Female pronoun count after rebalancing: ' + str(female_pronoun_count))
    
    return data

In [53]:
woman_keywords = ['woman', 'women', 'female', 'she', 'her', 'hers']
man_keywords = ['man', 'men', 'male', 'he', 'his', 'him']
CONTENT_COLUMN_NAME = 'content'


def shuffle_data(df, seed=0):
	return df.sample(frac=1, random_state=0)

def parse_args():
	parser = argparse.ArgumentParser(description='Process input files into train, dev, test')
	parser.add_argument('-i', dest='input_uri', type=str)
	parser.add_argument('-o', dest='output_dir', type=str)
	# Whether to filter for only sentences with a gender pronoun present 
	parser.add_argument('--filter_gender', default=False, action=argparse.BooleanOptionalAction)
	# This is a hyperparameter used to tune how many rows to drop in order to achieve roughly ~50/50 pronoun balance
	parser.add_argument('-r', dest='ratio_param', type=str, default=1.35)
	return parser

def filter_gender_entries(data):
	gender_keyword_filtered_df = data[data[CONTENT_COLUMN_NAME + "_list"].apply(lambda x: any([k in x for k in woman_keywords]))]
	gender_keyword_filtered_df = gender_keyword_filtered_df[gender_keyword_filtered_df[CONTENT_COLUMN_NAME + "_list"].apply(lambda x: any([k in x for k in man_keywords]))]
	return gender_keyword_filtered_df

# TODO: Make this more systematic rather than an approximation
def proportion_gender_entries(data, male_pronoun_count, female_pronoun_count, ratio):
	women = data[data['content_list'].apply(lambda x: any([k in x for k in woman_keywords]))]
	men = data[data['content_list'].apply(lambda x: any([k in x for k in man_keywords]))]
	men_with_women = men[men['content_list'].apply(lambda x: any([k in x for k in woman_keywords]))]
	men_no_women = men[men['content_list'].apply(lambda x: any([k in x for k in woman_keywords]) == False)]

	remove_n = int((male_pronoun_count - female_pronoun_count) / ratio)
	drop_indices = np.random.choice(men_no_women.index, remove_n, replace=False)
	men_no_women_subset = men_no_women.drop(drop_indices)
	rebalanced = pd.concat([women, men_no_women_subset], axis=0)
	return rebalanced

def count_gender_entries(data):
	series = pd.Series(Counter([y for x in data[CONTENT_COLUMN_NAME + "_list"] for y in x]))
	female_pronoun_count = 0
	male_pronoun_count = 0 
	for female_pronoun in woman_keywords:
		female_pronoun_count += series[female_pronoun]
	for male_pronoun in man_keywords:
		male_pronoun_count += series[male_pronoun]

	return female_pronoun_count, male_pronoun_count